## Import Library

In [1]:
from machine_lib import * 

## 1, Login
在machine_lib文件的login方法中填写用户名和密码后保存然后来到本文件Restart Kernal后重新import machine_lib

In [13]:
s = login()

b'{"user":{"id":"WM44289"},"token":{"expiry":14400.0},"permissions":["CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","SUPER_ALPHA","VISUALIZATION","WORKDAY"]}'


## 2, get data fields

In [14]:
df = get_datafields(s, dataset_id = 'pv1', region='USA', universe='TOP3000', delay=1)
print(df)

             id                                description  \
0         adv20       Average daily volume in past 20 days   
1           cap  Daily market capitalization (in millions)   
2         close                          Daily close price   
3       country                           Country grouping   
4      dividend                                   Dividend   
5      exchange                          Exchange grouping   
6          high                           Daily high price   
7      industry                          Industry grouping   
8           low                            Daily low price   
9        market                            Market grouping   
10         open                           Daily open price   
11      returns                              Daily returns   
12       sector                            Sector grouping   
13    sharesout     Daily outstanding shares (in millions)   
14        split                          Stock split ratio   
15  subi

### Matrix Data

In [4]:
print(df[df['type'] == "MATRIX"]["id"].tolist())
pc_fields = process_datafields(df, "matrix")
print(pc_fields)

['mdl26_5yr_hstrcl_grwth_rt', 'mdl26_7dy_bld_stmt_flg_fq1_rnngs', 'mdl26_actual_last_y_earnings', 'mdl26_blndd_grwth_ndstry_prcntl', 'mdl26_blndd_grwth_rt_smrtstmt', 'mdl26_blndd_grwth_sctr_prcntl', 'mdl26_chng_frm_52wk_hgh_prc', 'mdl26_chng_frm_52wk_lw_prc', 'mdl26_chng_rltv_t_cc_40_30', 'mdl26_chng_rltv_t_cntry_bnchmrk_30', 'mdl26_chng_rltv_t_dj_stxx_600_30', 'mdl26_chng_rltv_t_fts_100_30', 'mdl26_chng_rltv_t_hng_sng_30', 'mdl26_chng_rltv_t_js_ll_shr_30', 'mdl26_chng_rltv_t_msc_f_30', 'mdl26_chng_rltv_t_msc_mf_30', 'mdl26_chng_rltv_t_msc_pc_x_jp_30', 'mdl26_chng_rltv_t_msc_rp_30', 'mdl26_chng_rltv_t_ndstry_bnchmrk_30', 'mdl26_chng_rltv_t_r1000_grwth_30', 'mdl26_chng_rltv_t_rssll_2000_30', 'mdl26_chng_rltv_t_rssll_mdcp_30', 'mdl26_chng_rltv_t_sp_400_30', 'mdl26_chng_rltv_t_sp_500_30', 'mdl26_chng_rltv_t_sp_600_30', 'mdl26_chng_rltv_t_sx_100_30', 'mdl26_chng_rltv_t_sx_200_30', 'mdl26_chng_rltv_t_sx_300_30', 'mdl26_chng_rltv_t_sx_ll_rd_30', 'mdl26_chng_rltv_t_tpx_30', 'mdl26_chng_rltv_t

### Vector Data

In [6]:
df = get_datafields(s, dataset_id='news85', region='USA', universe='TOP3000', delay=1)
print(df[df['type'] == "VECTOR"]["id"].tolist())
print(process_datafields(df, "vector"))

['mws85_sentiment']
['winsorize(ts_backfill(vec_avg(mws85_sentiment), 120), std=4)', 'winsorize(ts_backfill(vec_sum(mws85_sentiment), 120), std=4)', 'winsorize(ts_backfill(vec_ir(mws85_sentiment), 120), std=4)', 'winsorize(ts_backfill(vec_max(mws85_sentiment), 120), std=4)', 'winsorize(ts_backfill(vec_count(mws85_sentiment), 120), std=4)', 'winsorize(ts_backfill(vec_skewness(mws85_sentiment), 120), std=4)', 'winsorize(ts_backfill(vec_stddev(mws85_sentiment), 120), std=4)', 'winsorize(ts_backfill(vec_choose(mws85_sentiment, nth=-1), 120), std=4)', 'winsorize(ts_backfill(vec_choose(mws85_sentiment, nth=0), 120), std=4)']


## 3, Alpha factory
### start with First Order

In [8]:
first_order = get_first_order(pc_fields, ts_ops)
print(first_order[:10])
print(len(first_order))

['winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4)', 'ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 5)', 'ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 22)', 'ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 66)', 'ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 120)', 'ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 240)', 'ts_max(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 5)', 'ts_max(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 22)', 'ts_max(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 66)', 'ts_max(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 120)']
11448


In [11]:
# Pad initial decay with alpha
init_decay = 6
fo_alpha_list = []
for alpha in first_order:
    fo_alpha_list.append((alpha, init_decay))
print(len(fo_alpha_list))
print(fo_alpha_list[:5])

11448
[('winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4)', 6), ('ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 5)', 6), ('ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 22)', 6), ('ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 66)', 6), ('ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 120)', 6)]


In [10]:
# Load alphas to task pools
pools = load_task_pool(fo_alpha_list, 9, 10)
print(pools[0])

[[('winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4)', 6), ('ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 5)', 6), ('ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 22)', 6), ('ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 66)', 6), ('ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 120)', 6), ('ts_min(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 240)', 6), ('ts_max(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 5)', 6), ('ts_max(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 22)', 6), ('ts_max(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 66)', 6), ('ts_max(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 120)', 6)], [('ts_max(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 240)', 6), ('ts_rank(winsorize(ts_backfill(mdl26_5yr_hstrcl_grwth_rt, 120), std=4), 5)', 6), ('ts_rank(winsorize(ts

## 4, simulate alphas

In [ ]:
# Simulate First Order
multi_simulate(pools, "SUBINDUSTRY", "CHN", "TOP2000U", 0)

## 5, Select alphas
go to web alphas penal to look for the number and date to track for next order improve

In [ ]:
## get promising alphas to improve in the next order
fo_tracker = get_alphas("06-06", "06-08", 1.2, 0.5, "CHN", 320, "track")

## 6, Next order improvement - Second Order
second order: ts_ops(field, days) -> group_ops(ts_ops(field, days), group)

In [4]:
print(len(fo_tracker['next']))
print(len(fo_tracker['decay']))

fo_layer = transform(fo_tracker['next'] + fo_tracker['decay'], 'chn')
so_alpha_list = []
group_ops = group_ops = ["group_neutralize", "group_rank", "group_normalize", "group_scale", "group_zscore"]
for region, couples in fo_layer.items():
    for expr, decay in couples:
        for alpha in get_group_second_order_factory([expr], group_ops, region):
            so_alpha_list.append((alpha,decay))

print(len(so_alpha_list))
print(so_alpha_list[:3])

364
36
58000


### Simulate second order

In [7]:
so_pools = load_task_pool(so_alpha_list, 9, 10)
multi_simulate(so_pools, 'SUBINDUSTRY', 'CHN', 'TOP2000U', 0)

b'{"user":{"id":"WM44289"},"token":{"expiry":14400.0},"permissions":["CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","SUPER_ALPHA","VISUALIZATION","WORKDAY"]}'
pool 0 task 9 post done
pool 0 task 9 simulate done
pool 1 task 9 post done
pool 1 task 9 simulate done
pool 2 task 9 post done
pool 2 task 9 simulate done
pool 3 task 9 post done
pool 3 task 9 simulate done
pool 4 task 9 post done
pool 4 task 9 simulate done
pool 5 task 9 post done
pool 5 task 9 simulate done
pool 6 task 9 post done
pool 6 task 9 simulate done
pool 7 task 9 post done
pool 7 task 9 simulate done
pool 8 task 9 post done
pool 8 task 9 simulate done
pool 9 task 9 post done
pool 9 task 9 simulate done
pool 10 task 9 post done
pool 10 task 9 simulate done
pool 11 task 9 post done
pool 11 task 9 simulate done
pool 12 task 9 post done
pool 12 task 9 simulate done
pool 13 task 9 post done
pool 13 task 9 simulate done
pool 14 task 9 post done
pool 14 task 9 simulate done
pool 15 task 9 post done
pool 15 task 9 s

## Higher Order for improvement - Third Order
group_ops(ts_ops(field, days), group) -> trade_when(entre_event, group_ops(ts_ops(field, days), group), exit_event)

In [ ]:
## get promising alphas from second order to improve in the third order
so_tracker = get_alphas("06-06", "06-08", 1.4, 0.7, "CHN", 360, "track")

print(len(so_tracker['next']))
print(len(so_tracker['decay']))

so_layer = transform(so_tracker['next'] + so_tracker['decay'], 'chn')
th_alpha_list = []
for region, couples in so_layer.items():
    for expr, decay in couples:
        for alpha in trade_when_factory("trade_when",expr,region):
            th_alpha_list.append((alpha,decay))
print(len(th_alpha_list))

### Simulate Third Order

In [ ]:
# Simulate third order
th_pools = load_task_pool(th_alpha_list, 9, 10)
multi_simulate(th_pools, 'SUBINDUSTRY', 'CHN', 'TOP2000U', 0)

## 7, Get submittable alphas
check_submission and view alphas

In [ ]:
# get submitable alphas to check submission
th_tracker = get_alphas("07-16", "07-20", 1.58, 1, "CHN", 200, "submit")

In [ ]:
stone_bag = []
for alpha in th_tracker['next'] + th_tracker['decay']:
    stone_bag.append(alpha[0])
print(len(stone_bag))
gold_bag = []
check_submission(stone_bag, gold_bag, 0)

In [ ]:
# look date and metrics to locate alphas in the web
view_alphas(gold_bag)

## 8, fine-tune submittable alphas
neutralization, performance comparison, turnover, margin